# TV Script Generation

In this project, you'll generate your own [Seinfeld](https://en.wikipedia.org/wiki/Seinfeld) TV scripts using RNNs.  You'll be using part of the [Seinfeld dataset](https://www.kaggle.com/thec03u5/seinfeld-chronicles#scripts.csv) of scripts from 9 seasons.  The Neural Network you'll build will generate a new ,"fake" TV script, based on patterns it recognizes in this training data.

## Get the Data

The data is already provided for you in `./data/Seinfeld_Scripts.txt` and you're encouraged to open that file and look at the text. 
>* As a first step, we'll load in this data and look at some samples. 
* Then, you'll be tasked with defining and training an RNN to generate a new script!

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# load in data
import helper
data_dir = './data/Seinfeld_Scripts.txt'
text = helper.load_data(data_dir)

## Explore the Data
Play around with `view_line_range` to view different parts of the data. This will give you a sense of the data you'll be working with. You can see, for example, that it is all lowercase text, and each new line of dialogue is separated by a newline character `\n`.

In [2]:
# type()  # str
text[:100].split()   # line is marked by    \n ; there are so many long and short lines; its just    \n   that decides

['jerry:',
 'do',
 'you',
 'know',
 'what',
 'this',
 'is',
 'all',
 'about?',
 'do',
 'you',
 'know,',
 'why',
 'were',
 'here?',
 'to',
 'be',
 'out,',
 'this',
 'is',
 'out...and']

In [3]:
view_line_range = (10, 25)    # This is a tuple

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
# roughly because the puntation marks are often adjoined to words

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 5.544240293684143

The lines 10 to 25:
jerry: oh, you dont recall? 

george: (on an imaginary microphone) uh, no, not at this time. 

jerry: well, senator, id just like to know, what you knew and when you knew it. 

claire: mr. seinfeld. mr. costanza. 

george: are, are you sure this is decaf? wheres the orange indicator? 

claire: its missing, i have to do it in my head decaf left, regular right, decaf left, regular right...its very challenging work. 

jerry: can you relax, its a cup of coffee. claire is a professional waitress. 

claire: trust me george. no one has any interest in seeing you on caffeine. 


---
## Implement Pre-processing Functions
The first thing to do to any dataset is pre-processing.  Implement the following pre-processing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following **tuple** `(vocab_to_int, int_to_vocab)`

In [4]:
vocab_list =  list(set(text.split()))
vocab_to_int = {word: ii for ii, word in enumerate(vocab_list, start=1)}
int_to_vocab = {ii: word for word, ii in vocab_to_int.items()}

In [5]:
len(int_to_vocab)

46367

In [6]:
import problem_unittests as tests
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    
    ## TODO:
#     text = text.lower()
#     text = ''.join([c for c in text if c not in punctuation])
#     text = ''.join([each_dialogue for each_dialogue in text.split('\n\n')])
#    Note: that we already have the input to this function in list form

    word_counts = Counter(text)
    count_words_sorted = sorted(word_counts, key=word_counts.get, reverse=True)    
    
    # TODO: Implement Function
    vocab_to_int = {word: ii for ii, word in enumerate(count_words_sorted,1)}
    int_to_vocab = {ii: word for word, ii in vocab_to_int.items()}  
    
    # return tuple
    return (vocab_to_int, int_to_vocab)



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks can create multiple ids for the same word. For example, "bye" and "bye!" would generate two different word ids.

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( **.** )
- Comma ( **,** )
- Quotation Mark ( **"** )
- Semicolon ( **;** )
- Exclamation mark ( **!** )
- Question mark ( **?** )
- Left Parentheses ( **(** )
- Right Parentheses ( **)** )
- Dash ( **-** )
- Return ( **\n** )

This dictionary will be used to tokenize the symbols and add the delimiter (space) around it.  This separates each symbols as its own word, making it easier for the neural network to predict the next word. Make sure you don't use a value that could be confused as a word; for example, instead of using the value "dash", try using something like "||dash||".

In [7]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    ## Note: it does not require any input its just to return a dictorty that maps the above
    my_dict = {
        '.': "||period||",
        ',': "||Comma||",
        '"': "||QuotationMark||",
        ';': "||Semicolon||",
        '!': "||ExclamationMark||",
        '?': "||QuestionMark||",
        '(': "||LeftParentheses||",
        ')': "||RightParentheses||",
        '-': "||Dash||",
        '\n': "||Return||"        
    }
    return my_dict

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Pre-process all the data and save it

Running the code cell below will pre-process all the data and save it to file. You're encouraged to lok at the code for `preprocess_and_save_data` in the `helpers.py` file to see what it's doing in detail, but you do not need to change this code.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# pre-process training data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

NameError: name 'helper' is not defined

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

In [ ]:
# print(type(int_text))     # <class 'list'>
# print(len(int_text))      # 892110
# print(type(vocab_to_int)) # <class 'dict'>
# print(len(vocab_to_int))  # 21388
# print(type(int_to_vocab)) # <class 'dict'>
# print(len(int_to_vocab))  # 21388
# print(type(token_dict))   # <class 'dict'>
# print(len(token_dict))    # 10

# vocab_to_int # this includes the punctuations which was defined as a dictorary as a dataloader

## Build the Neural Network
In this section, you'll build the components necessary to build an RNN by implementing the RNN Module and forward and backpropagation functions.

### Check Access to GPU

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch

# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')
    
import numpy as np


## Input
Let's start with the preprocessed input data. We'll use [TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset) to provide a known format to our dataset; in combination with [DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader), it will handle batching, shuffling, and other dataset iteration functions.

You can create data with TensorDataset by passing in feature and target tensors. Then create a DataLoader as usual.
```
data = TensorDataset(feature_tensors, target_tensors)
data_loader = torch.utils.data.DataLoader(data, 
                                          batch_size=batch_size)
```

### Batching
Implement the `batch_data` function to batch `words` data into chunks of size `batch_size` using the `TensorDataset` and `DataLoader` classes.

>You can batch words using the DataLoader, but it will be up to you to create `feature_tensors` and `target_tensors` of the correct size and content for a given `sequence_length`.

For example, say we have these as input:
```
words = [1, 2, 3, 4, 5, 6, 7]
sequence_length = 4
```

Your first `feature_tensor` should contain the values:
```
[1, 2, 3, 4]
```
And the corresponding `target_tensor` should just be the next "word"/tokenized word value:
```
5
```
This should continue with the second `feature_tensor`, `target_tensor` being:
```
[2, 3, 4, 5]  # features
6             # target
```

In [3]:
from torch.utils.data import TensorDataset, DataLoader


def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    # TODO: Implement function
    
    #step 1: Decide length of the words list: remainder based on sequence length
    #step 2: translate for 1 unit across the entire row
    # Manual data for dataloader: data, labels: https://datascience.stackexchange.com/questions/45916/loading-own-train-data-and-labels-in-dataloader-using-pytorch
    # To convert into tensor form: https://stackoverflow.com/questions/44429199/how-to-load-a-list-of-numpy-arrays-to-pytorch-dataset-loader
            
    print(type(words))  # Thats how I know this is a list and not np.array
    feature = []  # Not an np.array([])
    target =  []  # Not an np.array([])
    
    n_targets = (len(words) - (sequence_length)) # Not (sequence_length + 1) because the size of the label is 1 that gets subtracted 
    
    for i in range(n_targets):
        feature.append(words[i:i+sequence_length]) 
        target.append(words[i+sequence_length])              
        
    feature_tensors, target_tensors = torch.from_numpy(np.array(feature)), torch.from_numpy(np.array(target))
    # otherway to get the train data is using a tuple
    
    
    # create dataloader
    data = TensorDataset(feature_tensors, target_tensors)    
    dataloader = DataLoader(data, batch_size=batch_size, shuffle=True)
        
    # return a dataloader
    return dataloader

### Test your dataloader 

You'll have to modify this code to test a batching function, but it should look fairly similar.

Below, we're generating some test text data and defining a dataloader using the function you defined, above. Then, we are getting some sample batch of inputs `sample_x` and targets `sample_y` from our dataloader.

Your code should return something like the following (likely in a different order, if you shuffled your data):

```
torch.Size([10, 5])
tensor([[ 28,  29,  30,  31,  32],
        [ 21,  22,  23,  24,  25],
        [ 17,  18,  19,  20,  21],
        [ 34,  35,  36,  37,  38],
        [ 11,  12,  13,  14,  15],
        [ 23,  24,  25,  26,  27],
        [  6,   7,   8,   9,  10],
        [ 38,  39,  40,  41,  42],
        [ 25,  26,  27,  28,  29],
        [  7,   8,   9,  10,  11]])

torch.Size([10])
tensor([ 33,  26,  22,  39,  16,  28,  11,  43,  30,  12])
```

### Sizes
Your sample_x should be of size `(batch_size, sequence_length)` or (10, 5) in this case and sample_y should just have one dimension: batch_size (10). 

### Values

You should also notice that the targets, sample_y, are the *next* value in the ordered test_text data. So, for an input sequence `[ 28,  29,  30,  31,  32]` that ends with the value `32`, the corresponding output should be `33`.

In [4]:
# test dataloader

test_text = range(50) # np.arange(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

<class 'range'>
torch.Size([10, 5])
tensor([[ 17,  18,  19,  20,  21],
        [ 37,  38,  39,  40,  41],
        [ 23,  24,  25,  26,  27],
        [ 18,  19,  20,  21,  22],
        [ 13,  14,  15,  16,  17],
        [ 34,  35,  36,  37,  38],
        [ 32,  33,  34,  35,  36],
        [  2,   3,   4,   5,   6],
        [  8,   9,  10,  11,  12],
        [ 40,  41,  42,  43,  44]])

torch.Size([10])
tensor([ 22,  42,  28,  23,  18,  39,  37,   7,  13,  45])


---
## Build the Neural Network
Implement an RNN using PyTorch's [Module class](http://pytorch.org/docs/master/nn.html#torch.nn.Module). You may choose to use a GRU or an LSTM. To complete the RNN, you'll have to implement the following functions for the class:
 - `__init__` - The initialize function. 
 - `init_hidden` - The initialization function for an LSTM/GRU hidden state
 - `forward` - Forward propagation function.
 
The initialize function should create the layers of the neural network and save them to the class. The forward propagation function will use these layers to run forward propagation and generate an output and a hidden state.

**The output of this model should be the *last* batch of word scores** after a complete sequence has been processed. That is, for each input sequence of words, we only want to output the word scores for a single, most likely, next word.

### Hints

1. Make sure to stack the outputs of the lstm to pass to your fully-connected layer, you can do this with `lstm_output = lstm_output.contiguous().view(-1, self.hidden_dim)`
2. You can get the last batch of word scores by shaping the output of the final, fully-connected layer like so:

```
# reshape into (batch_size, seq_length, output_size)
output = output.view(batch_size, -1, self.output_size)
# get last batch
out = output[:, -1]
```

In [5]:
import torch.nn as nn

class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(RNN, self).__init__()
        # TODO: Implement function
        
        # set class variables
        self.vocab_size = vocab_size
        self.output_size = output_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.dropout = dropout
        ### https://docs.python.org/3/tutorial/classes.html
        
        self.embed= nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)
        
        self.lstm = nn.LSTM(input_size = embedding_dim, hidden_size=hidden_dim, num_layers=n_layers,
                            dropout=dropout, batch_first=True)
        
        self.dropout = nn.Dropout(dropout)
        
        self.fc = nn.Linear(hidden_dim, output_size)
        # define model layers
    
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """
        # TODO: Implement function   
        #### define the bacth size here
        
        batch_size = nn_input.size(0)
        
        nn_input = nn_input.long()              # converts into a long tensor # https://discuss.pytorch.org/t/how-to-conver-a-floattensor-to-longtensor/22173/10
        nn_input_embed = self.embed(nn_input)
        
        lstm_output, hidden = self.lstm(nn_input_embed, hidden)
         
        lstm_output = lstm_output.contiguous().view(-1, self.hidden_dim)
        
        out = self.dropout(lstm_output)
        out = self.fc(out)               
        
        # reshape into (batch_size, seq_length, output_size)
        out = out.view(batch_size, -1, self.output_size) 
        #https://stackoverflow.com/questions/42866654/what-is-the-difference-between-view-and-unsqueeze-in-torch
        # the above link clarifies .unsqueeze() vs .view()
        # get last batch
        out = out[:, -1]

        # return one batch of output word scores and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        # Implement function
        weight = next(self.parameters()).data
        
        # initialize hidden state with zero weights, and move to GPU if available
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
            ## Note: batch_size does not change with each updated iteration, so no need to declare is as self.batch_size
            ## I am not sure if the above reason is correct, plz check
            
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
    
        return hidden

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_rnn(RNN, train_on_gpu)

Tests Passed


### Define forward and backpropagation

Use the RNN class you implemented to apply forward and back propagation. This function will be called, iteratively, in the training loop as follows:
```
loss = forward_back_prop(decoder, decoder_optimizer, criterion, inp, target)
```

And it should return the average loss over a batch and the hidden state returned by a call to `RNN(inp, hidden)`. Recall that you can get this loss by computing it, as usual, and calling `loss.item()`.

**If a GPU is available, you should move your data to that GPU device, here.**

In [6]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
    """
    Forward and backward propagation on the neural network
    :param decoder: The PyTorch Module that holds the neural network
    :param decoder_optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """
    
    # TODO: Implement Function
    
    # move data to GPU, if available
    if (train_on_gpu):
        inp, target = inp.cuda(), target.cuda()
        
    # perform backpropagation and optimization
    hidden = tuple([each.data for each in hidden])
    
    # zero out the accumilated gradients
    rnn.zero_grad()
    # forward propogation
    output, hidden = rnn(inp, hidden) 
    ## print(type(inp))     # <class 'torch.Tensor'>
    ## print(type(hidden))  # <class 'tuple'>
    
    ## print(type(output))  # <class 'torch.Tensor'>
    ## loss = criterion(output.squeeze(), labels.float()) : from sentiment analysis notebook
    ## print(type(target))  # <class 'torch.Tensor'>
    
    # loss function implement
    loss = criterion(output, target)  
    ## print(type(loss))    # <class 'torch.Tensor'>
    ## print(type(loss.item()))  # <class 'float'>
    ### Note that 'loss.item()' ensures that this is in float type: this is essential for computing the average loss in float
    
    # clipping of gradients to prevent exploding
    nn.utils.clip_grad_norm_(rnn.parameters(), 5)
    # backpropogate losses
    loss.backward()
    # update the weights
    optimizer.step()
    
    # return the loss over a batch and the hidden state produced by our model
    return loss.item(), hidden  

# Note that these tests aren't completely extensive.
# they are here to act as general checks on the expected outputs of your functions
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_forward_back_prop(RNN, forward_back_prop, train_on_gpu)

Tests Passed


## Neural Network Training

With the structure of the network complete and data ready to be fed in the neural network, it's time to train it.

### Train Loop

The training loop is implemented for you in the `train_decoder` function. This function will train the network over all the batches for the number of epochs given. The model progress will be shown every number of batches. This number is set with the `show_every_n_batches` parameter. You'll set this parameter along with other parameters in the next section.

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}\n'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []

    # returns a trained rnn
    return rnn

### Hyperparameters

Set and train the neural network with the following parameters:
- Set `sequence_length` to the length of a sequence.
- Set `batch_size` to the batch size.
- Set `num_epochs` to the number of epochs to train for.
- Set `learning_rate` to the learning rate for an Adam optimizer.
- Set `vocab_size` to the number of uniqe tokens in our vocabulary.
- Set `output_size` to the desired size of the output.
- Set `embedding_dim` to the embedding dimension; smaller than the vocab_size.
- Set `hidden_dim` to the hidden dimension of your RNN.
- Set `n_layers` to the number of layers/cells in your RNN.
- Set `show_every_n_batches` to the number of batches at which the neural network should print progress.

If the network isn't getting the desired results, tweak these parameters and/or the layers in the `RNN` class.

In [9]:
# Data params
# Sequence Length
sequence_length = 25 # of words in a sequence
# Batch Size
batch_size = 50

# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

<class 'list'>


In [8]:
# Training parameters
# Number of Epochs
num_epochs = 4
# Learning Rate
learning_rate = 0.001

# Model parameters
# Vocab size
vocab_size = 21389
# Output size
output_size = 21389
# Embedding Dimension
embedding_dim = 400
# Hidden Dimension
hidden_dim = 256
# Number of RNN Layers
n_layers = 3

# Show stats for every n number of batches
show_every_n_batches = 500


#### check: so far even after using the emdedding; my vocab_size and the output_size are the same
#### This clue is based on the logic used for char level rnn next char generation

### Train
In the next cell, you'll train the neural network on the pre-processed data.  If you have a hard time getting a good loss, you may consider changing your hyperparameters. In general, you may get better results with larger hidden and n_layer dimensions, but larger models take a longer time to train. 
> **You should aim for a loss less than 3.5.** 

You should also experiment with different sequence lengths, which determine the size of the long range dependencies that a model can learn.

In [10]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
helper.save_model('./save/trained_rnn_outputsize21389_seq25_nlayers3_hiddendim256_epochs4', trained_rnn)  # n_layers = 3
## my next attempt 1 epoch and seq length of 25 or n_layers = 3
print('Model Trained and Saved')

Training for 4 epoch(s)...
Epoch:    1/4     Loss: 6.025237840652466

Epoch:    1/4     Loss: 5.2932371902465825

Epoch:    1/4     Loss: 5.084957532405853

Epoch:    1/4     Loss: 5.0400259981155395

Epoch:    1/4     Loss: 4.985714210510254

Epoch:    1/4     Loss: 4.915093023300171

Epoch:    1/4     Loss: 4.887238888263703

Epoch:    1/4     Loss: 4.828061000347137

Epoch:    1/4     Loss: 4.836341613769531

Epoch:    1/4     Loss: 4.7723683223724365

Epoch:    1/4     Loss: 4.772382217407227

Epoch:    1/4     Loss: 4.718476309776306

Epoch:    1/4     Loss: 4.659194873332977

Epoch:    1/4     Loss: 4.716630287647248

Epoch:    1/4     Loss: 4.63399978017807

Epoch:    1/4     Loss: 4.644988305568695

Epoch:    1/4     Loss: 4.642513786315918

Epoch:    1/4     Loss: 4.599945190429687

Epoch:    1/4     Loss: 4.585137785434723

Epoch:    1/4     Loss: 4.544533824920654

Epoch:    1/4     Loss: 4.547387892246246

Epoch:    1/4     Loss: 4.553623499870301

Epoch:    1/4     Loss: 4

/opt/conda/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Model Trained and Saved


In [ ]:
# ./save/trained_rnn_outputsize21389_seq25_nlayers3_hiddendim256_epochs4
# it starts with high error, 
# note that sequence length = 25; now can expect ca. 70 subparts, earlier 35 subparts;  takes 30 seconds each current sub part 
# earlier:  sequence length = 50 ; earlier ca. 35 subparts;                             takes 53 sec each sub part   
# the time taken explains the time for the dataloader to load data and print etc so saves 7 seconds per 50 words of training
# convergence is reallly great with sequence legnth of 25 during 1st epoch; remember starts with high error; not yet celabrate
# I expect 35 min for each full epoch and then 4 such epochs

# = 50seq leng *500show_every_n_batches *35subparts      =  875,000; now lets check the no of words in the 
# print(len(int_text))      # 892110                     =  892,110 which matches,   WOOOW HOORAY

# Epoch:    1/4     Loss: 6.025237840652466   (time stamp: 24min 40 sec remaning)
# Epoch:    1/4     Loss: 5.2932371902465825
# Epoch:    1/4     Loss: 5.084957532405853   (time stamp: 23min 41 sec remaning)
# Epoch:    1/4     Loss: 5.0400259981155395  (time stamp: 24min 08 (approx) sec remaning)
# Epoch:    1/4     Loss: 4.985714210510254   (time stamp: 23min 41 sec remaning)
# Epoch:    1/4     Loss: 4.915093023300171   (time stamp: 23min 11 sec remaning)
# ...........
# Epoch:    1/4     Loss: 4.544533824920654   (this is 20 th subpart)
# ..........
# Epoch:    1/4     Loss: 4.49369328546524

# Epoch:    2/4     Loss: 4.347288231372834   (at end of 2nd epoch)
# Epoch:    4/4     Loss: 4.189169535636902   (at end of 4th epoch)

# I am confused on teh number of words it does not seem to match; reprint and chekc the total number of words

In [ ]:
## trained model name: ./save/trained_rnn
# each of the subpart within an epoch takes 2min 30sec
# hidden_dim = 256, n_layers = 2; seq length = 200 batch size = 50, output size = 21388  # epochs total of 5


### default params
# Training for 5 epoch(s)...
# Epoch:    1/5     Loss: 5.679670822143555
# Epoch:    1/5     Loss: 5.0699342608451845
# Epoch:    1/5     Loss: 4.947699316501618
# Epoch:    1/5     Loss: 4.853958524703979
# Epoch:    1/5     Loss: 4.814269760131836
# Epoch:    1/5     Loss: 4.719399832248688
# Epoch:    1/5     Loss: 4.7092023739814755
# Epoch:    1/5     Loss: 4.679473986148834
# Epoch:    1/5     Loss: 4.668082592964172
# Epoch:    1/5     Loss: 4.651407203674316
# Epoch:    1/5     Loss: 4.60288259601593
# Epoch:    1/5     Loss: 4.566760490894318

##############################################################################
### default params + output size = 21380 ;
# './save/trained_rnn_outputsize21380' #  the code does work despite input and output size are not equal
## overwritten above file does not exist
# Epoch:    1/5     Loss: 5.68934774684906
# Epoch:    1/5     Loss: 5.127966684818268
# Epoch:    1/5     Loss: 5.0046741156578065
##############################################################################
## default params + input and output size = 21389 (one extra than vocab size for indiacting padding)
# ./save/trained_rnn_outputsize21389; seq length 50

# the time is a bit more than than 2.30 min for 4 times the seq length of 50, which is understandable
# Epoch:    1/5     Loss: 5.651392162799835
# Epoch:    1/5     Loss: 5.1205539383888246  (time stamp 4.15.. min remaining) 
# Epoch:    1/5     Loss: 4.966001293182373   (time stamp 3.20.. min remaining)
# Epoch:    1/5     Loss: 4.869546041488648   (time stamp 2.58 min remaining)
# Epoch:    1/5     Loss: 4.78015180683136    (time stamp 2.14 min reamining)     ca. 45 seconds per each part of epoch
# Epoch:    1/5     Loss: 4.72938712978363    (time stamp 1.29 min reamining)
# Epoch:    1/5     Loss: 4.750126441478729   (time stamp 0.46 min reamining)
# Epoch:    1/5     Loss: 4.63357901096344    (time stamp 0.03 min remaining)
# Epoch:    1/5     Loss: 4.650624115467071
# Epoch:    1/5     Loss: 4.63212097120285    (time stamp 3.34 min remaining)
# Epoch:    1/5     Loss: 4.62237011384964    (time stamp 2.49 min remaining)
# Epoch:    1/5     Loss: 4.543884534358979
# Epoch:    1/5     Loss: 4.551664614200592   (time stamp 1.22 min remaining)
# Epoch:    1/5     Loss: 4.568777572631836   ()   
# Epoch:    1/5     Loss: 4.568369558334351   (time stamp 4.52 min remaining) 
# Epoch:    1/5     Loss: 4.490623468399048   ()
# Epoch:    1/5     Loss: 4.504072373867035   (time stamp 3.24 min remaining) 
# Epoch:    1/5     Loss: 4.497775236129761   ()
# Epoch:    1/5     Loss: 4.491356122493744   (time stamp 1.56 min remaining)
# Epoch:    1/5     Loss: 4.460423276424408
# Epoch:    1/5     Loss: 4.459221571922302
# Epoch:    1/5     Loss: 4.455827499866485
# Epoch:    1/5     Loss: 4.447021509647369
# Epoch:    1/5     Loss: 4.445593259811401   
# Epoch:    1/5     Loss: 4.435606792926788
# Epoch:    1/5     Loss: 4.441254557609558
# Epoch:    1/5     Loss: 4.4385938363075255
# Epoch:    1/5     Loss: 4.45764255285263
# Epoch:    1/5     Loss: 4.370960216999054   It has gone past the limitation of getting stuck 
# Epoch:    1/5     Loss: 4.376053660392762
# Epoch:    1/5     Loss: 4.406688427448272
# Epoch:    1/5     Loss: 4.404595165252686
# Epoch:    1/5     Loss: 4.379129940509796
# Epoch:    1/5     Loss: 4.38213608121872
# Epoch:    1/5     Loss: 4.407978019237518
# so far we have seen 35 sub parts (of 44 sec each = 0.75 min each) of the 1st iteration: this is 26 min for each iteration  
# i.e. for all sub parts put togather
# lets check if its going through entire vocab: each sub part 'show_every_n_batches = 500 '  
# = 50seq leng *500show_every_n_batches *35subparts      =  875,000; now lets check the no of words in the 
# print(len(int_text))      # 892110                     =  892,110 which matches,   WOOOW HOORAY
 

# Epoch:    2/5     Loss: 4.337425182889106
# Epoch:    2/5     Loss: 4.292401518344879
# Epoch:    2/5     Loss: 4.259736950874329
# Epoch:    2/5     Loss: 4.291195661067962
# Epoch:    2/5     Loss: 4.273755095481873
# Epoch:    2/5     Loss: 4.311247765064239
# Epoch:    2/5     Loss: 4.270973267555237
# Epoch:    2/5     Loss: 4.274107063770294
# Epoch:    2/5     Loss: 4.272918271541595
# Epoch:    2/5     Loss: 4.306052016258239
# Epoch:    2/5     Loss: 4.279672818660736
# Epoch:    2/5     Loss: 4.238963483333587
# Epoch:    2/5     Loss: 4.313967868328095
# Epoch:    2/5     Loss: 4.308932987213135
# Epoch:    2/5     Loss: 4.2868383393287655
# Epoch:    2/5     Loss: 4.236069452762604
# Epoch:    2/5     Loss: 4.303145871639252
# Epoch:    2/5     Loss: 4.259615309238434
# Epoch:    2/5     Loss: 4.222451582431793
# Epoch:    2/5     Loss: 4.325679545402527
# Epoch:    2/5     Loss: 4.309269364833832
# Epoch:    2/5     Loss: 4.262269584655762
# Epoch:    2/5     Loss: 4.281542721748352
# Epoch:    2/5     Loss: 4.312215303897857
# Epoch:    2/5     Loss: 4.304422743797303
# Epoch:    2/5     Loss: 4.344115443706513
# Epoch:    2/5     Loss: 4.284948675632477
# Epoch:    2/5     Loss: 4.283864289760589
# Epoch:    2/5     Loss: 4.325867874145508
# Epoch:    2/5     Loss: 4.264327136039734
# Epoch:    2/5     Loss: 4.312139130115509
# Epoch:    2/5     Loss: 4.268797150611878
# Epoch:    2/5     Loss: 4.2740907907485965
# Epoch:    2/5     Loss: 4.280723660945893
# Epoch:    2/5     Loss: 4.297151346206665


# Epoch:    3/5     Loss: 4.199141204995009
# Epoch:    3/5     Loss: 4.210074543476105
# Epoch:    3/5     Loss: 4.1668775639534
# Epoch:    3/5     Loss: 4.190901988983154
# Epoch:    3/5     Loss: 4.217793397903442
# Epoch:    3/5     Loss: 4.17713164806366
# Epoch:    3/5     Loss: 4.208395316123962
# Epoch:    3/5     Loss: 4.197060702323913
# Epoch:    3/5     Loss: 4.219820072174072
# Epoch:    3/5     Loss: 4.213363020896912
# Epoch:    3/5     Loss: 4.173616827964783
# Epoch:    3/5     Loss: 4.160357120513916
# Epoch:    3/5     Loss: 4.197015605926514
# Epoch:    3/5     Loss: 4.212587597846984
# Epoch:    3/5     Loss: 4.246640689849854
# Epoch:    3/5     Loss: 4.207139207839965
# Epoch:    3/5     Loss: 4.221364508152008
# Epoch:    3/5     Loss: 4.182090219020844
# Epoch:    3/5     Loss: 4.205612687110901
# Epoch:    3/5     Loss: 4.195609215736389
# Epoch:    3/5     Loss: 4.202538077831268
# Epoch:    3/5     Loss: 4.219220636844635
# Epoch:    3/5     Loss: 4.24537016582489
# Epoch:    3/5     Loss: 4.245924527645111
# Epoch:    3/5     Loss: 4.223386630535126
# Epoch:    3/5     Loss: 4.2054250664711
# Epoch:    3/5     Loss: 4.221819386005402
# Epoch:    3/5     Loss: 4.243499981880188
# Epoch:    3/5     Loss: 4.220187591552734
# Epoch:    3/5     Loss: 4.256716556072235
# Epoch:    3/5     Loss: 4.240842217922211
# Epoch:    3/5     Loss: 4.2052885513305664
# Epoch:    3/5     Loss: 4.244597401618957
# Epoch:    3/5     Loss: 4.227527869224549
# Epoch:    3/5     Loss: 4.23251327085495
## convergence now is too slow:
###############################################################################
## next step increase n_hidden to 3 and seq length of 50 and then 25; note down time and see

## ??? try to estimate the no of times each epoch runs, depending on the batch_size

In [ ]:
## seq length = 50; n layers = 3 (this is the modfiction comapred to previous); n_hidden = 256; epochs 4
# ./save/trained_rnn_outputsize21389_seq50_nlayers3_epochs4
# in reality ran for 3 full epochs
# note: 53 sec for each subpart

# Epoch:    1/4     Loss: 5.969525673866272
# Epoch:    1/4     Loss: 5.235015101909638
# Epoch:    1/4     Loss: 5.139581728935242
# Epoch:    1/4     Loss: 5.051048342704773
# Epoch:    1/4     Loss: 4.958804966449738
# Epoch:    1/4     Loss: 4.935009340286255
# Epoch:    1/4     Loss: 4.881673234462738
# Epoch:    1/4     Loss: 4.823485461711884
# Epoch:    1/4     Loss: 4.809539026260376
# Epoch:    1/4     Loss: 4.757562675476074
# Epoch:    1/4     Loss: 4.754689996242523
# Epoch:    1/4     Loss: 4.693526710510254
# Epoch:    1/4     Loss: 4.65312957239151
# Epoch:    1/4     Loss: 4.685365090370178
# Epoch:    1/4     Loss: 4.643220893859863
# Epoch:    1/4     Loss: 4.636259063720703
# Epoch:    1/4     Loss: 4.601958547115326
# Epoch:    1/4     Loss: 4.591887588500977
# Epoch:    1/4     Loss: 4.609451609134674
# Epoch:    1/4     Loss: 4.634440176963806
# Epoch:    1/4     Loss: 4.548441838741303
# Epoch:    1/4     Loss: 4.538158977031708
# Epoch:    1/4     Loss: 4.548461719036102
# Epoch:    1/4     Loss: 4.528471171379089
# Epoch:    1/4     Loss: 4.554997947692871
# Epoch:    1/4     Loss: 4.515835035800934
# Epoch:    1/4     Loss: 4.491093781471252
# Epoch:    1/4     Loss: 4.491531324386597
# Epoch:    1/4     Loss: 4.508541049003601
# Epoch:    1/4     Loss: 4.518482370376587
# Epoch:    1/4     Loss: 4.487400938510895
# Epoch:    1/4     Loss: 4.432968014240265
# Epoch:    1/4     Loss: 4.450571044921875
# Epoch:    1/4     Loss: 4.472106902599335
# Epoch:    1/4     Loss: 4.465743690967559
        
        
# Epoch:    2/4     Loss: 4.39099864613854
# Epoch:    2/4     Loss: 4.326899797439575
# Epoch:    2/4     Loss: 4.373385711193085
# Epoch:    2/4     Loss: 4.397799228191376
# Epoch:    2/4     Loss: 4.356840434551239
# Epoch:    2/4     Loss: 4.34720047044754
# Epoch:    2/4     Loss: 4.354316328048706
# Epoch:    2/4     Loss: 4.369738773822784
# Epoch:    2/4     Loss: 4.350576617240906
# Epoch:    2/4     Loss: 4.359869847297668
# Epoch:    2/4     Loss: 4.37052705001831
# Epoch:    2/4     Loss: 4.37664156961441
# Epoch:    2/4     Loss: 4.350372472763062
# Epoch:    2/4     Loss: 4.337206182956695
# Epoch:    2/4     Loss: 4.302139759540558
# Epoch:    2/4     Loss: 4.316676615715027
# Epoch:    2/4     Loss: 4.314992161273956
# Epoch:    2/4     Loss: 4.3615400252342225
# Epoch:    2/4     Loss: 4.299490699291229
# Epoch:    2/4     Loss: 4.310936943054199
# Epoch:    2/4     Loss: 4.337585197925567
# Epoch:    2/4     Loss: 4.324730749607086
# Epoch:    2/4     Loss: 4.308525895118714
# Epoch:    2/4     Loss: 4.350326300621033
# Epoch:    2/4     Loss: 4.310077457427979
# Epoch:    2/4     Loss: 4.349040441513061
# Epoch:    2/4     Loss: 4.335028616428375
# Epoch:    2/4     Loss: 4.3154543356895445
# Epoch:    2/4     Loss: 4.298872980594635
# Epoch:    2/4     Loss: 4.371320405483246
# Epoch:    2/4     Loss: 4.3123153653144835
# Epoch:    2/4     Loss: 4.37046444940567
# Epoch:    2/4     Loss: 4.313201094150544
# Epoch:    2/4     Loss: 4.302883983612061
# Epoch:    2/4     Loss: 4.3122846159935

        
# Epoch:    3/4     Loss: 4.282588099752964
# Epoch:    3/4     Loss: 4.231969214916229
# Epoch:    3/4     Loss: 4.22475562953949
# Epoch:    3/4     Loss: 4.246066603183746
# Epoch:    3/4     Loss: 4.232581484317779
# Epoch:    3/4     Loss: 4.266102615833282
# Epoch:    3/4     Loss: 4.2714341073036195
# Epoch:    3/4     Loss: 4.234147319793701
# Epoch:    3/4     Loss: 4.200331241607666
# Epoch:    3/4     Loss: 4.279708036899566
# Epoch:    3/4     Loss: 4.203690019130707
# Epoch:    3/4     Loss: 4.2090207056999205
# Epoch:    3/4     Loss: 4.284964101314545
# Epoch:    3/4     Loss: 4.261759380340576
# Epoch:    3/4     Loss: 4.22664901304245
# Epoch:    3/4     Loss: 4.224100249767304
# Epoch:    3/4     Loss: 4.2329570908546446
# Epoch:    3/4     Loss: 4.2635151743888855
# Epoch:    3/4     Loss: 4.248309769153595
# Epoch:    3/4     Loss: 4.213542745113373
# Epoch:    3/4     Loss: 4.251161633968353
# Epoch:    3/4     Loss: 4.257901424407959
# Epoch:    3/4     Loss: 4.234765588283539
# Epoch:    3/4     Loss: 4.208155900001526
# Epoch:    3/4     Loss: 4.27323755979538
# Epoch:    3/4     Loss: 4.286443397521973
# Epoch:    3/4     Loss: 4.20337270450592
# Epoch:    3/4     Loss: 4.276894986152649
# Epoch:    3/4     Loss: 4.238702633857727
# Epoch:    3/4     Loss: 4.298445830821991
# Epoch:    3/4     Loss: 4.232857894420624
# Epoch:    3/4     Loss: 4.266783128261566
# Epoch:    3/4     Loss: 4.22976624917984
# Epoch:    3/4     Loss: 4.268706299781799
# Epoch:    3/4     Loss: 4.265986869335174
        
# Epoch:    4/4     Loss: 4.19936966669261
# Epoch:    4/4     Loss: 4.210239575386048
# Epoch:    4/4     Loss: 4.157375127315521

In [ ]:
## next sequence length 25; n layers = 3 and hidden dim = 256 ; slightly more time than periovus
## next sequence length 25; n layers = 3 and hidden dim = 512 ; about 30 sec more time than 44 sec = 66% increase

### Question: How did you decide on your model hyperparameters? 
For example, did you try different sequence_lengths and find that one size made the model converge faster? What about your hidden_dim and n_layers; how did you decide on those?

**Answer:** (Write answer, here)

---
# Checkpoint

After running the above training cell, your model will be saved by name, `trained_rnn`, and if you save your notebook progress, **you can pause here and come back to this code at another time**. You can resume your progress by running the next cell, which will load in our word:id dictionaries _and_ load in your saved model by name!

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
trained_rnn = helper.load_model('./save/trained_rnn')

## Generate TV Script
With the network trained and saved, you'll use it to generate a new, "fake" Seinfeld TV script in this section.

### Generate Text
To generate the text, the network needs to start with a single word and repeat its predictions until it reaches a set length. You'll be using the `generate` function to do this. It takes a word id to start with, `prime_id`, and generates a set length of text, `predict_len`. Also note that it uses topk sampling to introduce some randomness in choosing the most likely next word, given an output set of word scores!

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import torch.nn.functional as F

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        # the generated word becomes the next "current sequence" and the cycle can continue
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

### Generate a New Script
It's time to generate the text. Set `gen_length` to the length of TV script you want to generate and set `prime_word` to one of the following to start the prediction:
- "jerry"
- "elaine"
- "george"
- "kramer"

You can set the prime word to _any word_ in our dictionary, but it's best to start with a name for generating a TV script. (You can also start with any other names you find in the original text file!)

In [ ]:
# run the cell multiple times to get different results!
gen_length = 400 # modify the length to your preference
prime_word = 'jerry' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

#### Save your favorite scripts

Once you have a script that you like (or find interesting), save it to a text file!

In [ ]:
# save script to a text file
f =  open("generated_script_1.txt","w")
f.write(generated_script)
f.close()

# The TV Script is Not Perfect
It's ok if the TV script doesn't make perfect sense. It should look like alternating lines of dialogue, here is one such example of a few generated lines.

### Example generated script

>jerry: what about me?
>
>jerry: i don't have to wait.
>
>kramer:(to the sales table)
>
>elaine:(to jerry) hey, look at this, i'm a good doctor.
>
>newman:(to elaine) you think i have no idea of this...
>
>elaine: oh, you better take the phone, and he was a little nervous.
>
>kramer:(to the phone) hey, hey, jerry, i don't want to be a little bit.(to kramer and jerry) you can't.
>
>jerry: oh, yeah. i don't even know, i know.
>
>jerry:(to the phone) oh, i know.
>
>kramer:(laughing) you know...(to jerry) you don't know.

You can see that there are multiple characters that say (somewhat) complete sentences, but it doesn't have to be perfect! It takes quite a while to get good results, and often, you'll have to use a smaller vocabulary (and discard uncommon words), or get more data.  The Seinfeld dataset is about 3.4 MB, which is big enough for our purposes; for script generation you'll want more than 1 MB of text, generally. 

# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save another copy as an HTML file by clicking "File" -> "Download as.."->"html". Include the "helper.py" and "problem_unittests.py" files in your submission. Once you download these files, compress them into one zip file for submission.